In [1]:
#Script para referenciar artigos de espécies com avaliação do IUCN red list, usando a API de iucnredlist.org
#O script aqui não contém a chave da API, por questões de privacidade. Uma chave pode ser obtida em http://apiv3.iucnredlist.org/api/v3/token

import pywikibot
#from pywikibot import pagegenerators as pg
import sys
sys.path.append("/srv/paws/pwb/scripts")
import noreferences
from noreferences import NoReferencesBot
import mwparserfromhell
import requests
import re

In [3]:
#Lista gerada pelo Petscan cruzando items com identificador IUCN e artigos da pt.wikipedia.org marcados como sem referências
#species = input("Enter species name: ")
#print ("Species name is: " + species)filename = 'IUCNspecies.txt'
filename = 'IUCNspecies.txt'
binomial_list = 'binomial.txt'
with open(filename) as f:
    species_list = f.read().splitlines() 
#f = open('IUCNspecies.txt', 'r')
#species_list = f.readlines()
#f.close()

with open(binomial_list) as b:
    binomial = b.read().splitlines()
    

species_list =['Gaio-comum']
binomial = ['Garrulus glandarius']

print(species_list[0:10])
print(binomial[0:10])


['Gaio-comum']
['Garrulus glandarius']


In [4]:
pagename = species_list[0]
#site = pywikibot.Site('pt', 'wikipedia')
#site

def parse(title):
    """Parses templates in a page using mwparserfromhell"""
    site = pywikibot.Site('pt', 'wikipedia')
    page = pywikibot.Page(site, title)
    text = page.get()
    return mwparserfromhell.parse(text)

#wikicode = parse(pagename)

def iucn_ref(species, api_token):
    """Retrieves the reference for a IUCN red list species assessment and formats it using the Citar IUCN doi template"""
    citation_url = "http://apiv3.iucnredlist.org/api/v3/species/citation/" + species + "?token=" + api_token
    iucn_get = requests.get(citation_url)
    iucn_json = iucn_get.json()
    estado_ref = re.sub(r'(.+)\.? ([0-9]{4})\. ([A-Z][a-z]+) ([a-z]+)\. The IUCN Red List of Threatened Species ([0-9]{4}): (e\.T\S+)\. https:\/\/dx\.doi\.org\/(10\.2305\/IUCN\.UK\.\S+\.en) \.(?:(?:Downloaded)|(?:Accessed)) on ([0-9]{1,2} [A-Za-z]+ 202[1-9])', r'<ref name="IUCN">{{Citar IUCN doi|assessores=\1 |ano=\2 |género=\3 |espécie=\4 |anoIUCN=\5 |versão= \6|doi= \7|acessodata= 28 de outubro de 2021}}</ref>', iucn_json['result'][0]['citation'])
    return estado_ref

#estado_ref = iucn_ref(binomial[0], api_token)

def iucn_estado(species, api_token):
    """Retrieves the conservation status and the species authority from a IUCN red list species assessment"""
    estado_url = "http://apiv3.iucnredlist.org/api/v3/species/" + species + "?token=" + api_token
    iucn_get2 = requests.get(estado_url)
    iucn_json2 = iucn_get2.json()
    estado = iucn_json2['result'][0]['category']
    binomial_autoridade = iucn_json2['result'][0]['authority']
    return estado, binomial_autoridade

#estado, binomial_autoridade = iucn_estado(binomial[0], api_token)

#def iucn_insert(wikicode, estado_ref, estado, binomial_autoridade):
#    """Inserts new paramaters into Info/taxonomia template, based on information from IUCN"""
#    for template in wikicode.filter_templates():
#        if template.name.matches("Info/Taxonomia") and not template.has("estado"):
#            template.add("estado", estado)
#        elif template.name.matches("Info/Taxonomia") and not template.has("binomial_autoridade"):
#            template.add("binomial_autoridade", binomial_autoridade)
#        elif template.name.matches("Info/Taxonomia") and not template.has("estado"):
#            template.add("estado_ref", estado_ref)
    #template.remove("sem-fontes")
#    return wikicode

#wikicode = iucn_insert(wikicode, estado_ref, estado, binomial_autoridade)

#def save_page(wikicode, pagename):
#    site = pywikibot.Site('pt', 'wikipedia')
#    page = pywikibot.Page(site, pagename)
#    texto = page.get()
#    texto.text = wikicode
#    return texto.save()

#save_page(wikicode, pagename)

In [1]:
api_token = input("Enter IUCN token: ");

Enter IUCN token:  


In [7]:
index = 0
while index < len(species_list):
    pagename = species_list[index]
    #print(pagename)
    wikicode = parse(pagename)
    templates = wikicode.filter_templates()
    #print(templates)
    template = templates[1]
    #print(template.name)
    #print(template.params)
    #print(template.get("estado").value)
    estado_ref = iucn_ref(binomial[index], api_token)
    #print(estado_ref)
    estado, binomial_autoridade = iucn_estado(binomial[index], api_token)
    #print(estado)
    #print(binomial_autoridade)
    for template in wikicode.filter_templates():
        if template.name.matches("Info/Taxonomia") and not template.has("estado_ref"):
            template.add("estado_ref", estado_ref)
        if template.name.matches("Info/Taxonomia") and not template.has("estado"):
            template.add("estado", estado)
        if template.name.matches("Info/Taxonomia") and not template.has("binomial_autoridade"):
            template.add("binomial_autoridade", binomial_autoridade)
        if template.name.matches("Info/Taxonomia") and template.has("cor"):
            template.remove("cor")
        if template.name.matches("Info/Taxonomia") and template.has("estado"):
    #        if template.get("estado").value != estado:
             template.replace("estado", estado)
        if template.name.matches("Sem-fontes"):
            wikicode.remove(template)

    #wikicode = pywikibot.add_text(wikicode, "== Referências ==\n<references/>")
    #print(wikicode)

    #wikicode2 = iucn_insert(wikicode, estado_ref, estado, binomial_autoridade)
    #estado
    #print(wikicode)
    #print(wikicode2)
    #text = str(wikicode)
    #pagename.text = text
    #pagename.save()

    site = pywikibot.Site('pt', 'wikipedia')
    #site
    page = pywikibot.Page(site, pagename)
    #page
    text = page.get()
    #text
    #print(page.text)
    page_text = str(wikicode)

    if  page_text.find('<ref name=\"IUCN\"/>') == -1:
        regex = re.compile (r'^((?:[A-Z]|\').+?\.) ', re.M)
        page_text = re.sub(regex, '\g<1><ref name=\"IUCN\"/> ', page_text, 1)
        #page_text = re.sub(r'^((?:[A-Z]|\').+?\.)', '\g<1>'+ref_iucn, page_text)
    #print(page_text)

    bot = NoReferencesBot()
    page.text = bot.addReferences(page_text)

    print(page.text)
    #save_page(wikicode, pagename)
    page.save("adicionar referência, estado e binomial autoridade baseado em IUCNredlist")
    index += 1

Adding references section before Ligações externas section...



{{Insuficiente|1=Artigo [[WP:INSUFICIENTE]], por não possuir fonte nenhuma no seu artigo lusófono, descumprindo, portanto, [[WP:V]]. Esteve por anos marcado como tal com a [[Predefinição:Sem-fontes]], sem resultado.  Contudo, possui enorme [[WP:POTENCIAL]], por possuir entre 95 e 237 [[WP:IW|ligações interlínguas]]. Assim sendo, proponho [[WP:CANDIDATURA]].|2=[[Usuário(a):MisterSanderson|Mr. Sand.]] ❧ [[Usuário(a) Discussão:MisterSanderson|Ano ⓬]] 12h33min de 13 de novembro de 2021 (UTC)}}

{{Info/Taxonomia
| nome = Gaio-comum
| imagem = Garrulus glandarius B Luc Viatour.jpg
| imagem_legenda =Gaio-comum
| imagem2 = Garrulus glandarius iphigenia.ogg
| estado = LC
| reino = [[Animalia]]
| filo = [[Chordata]]
| classe = [[Aves]]
| ordem = [[Passeriformes]]
| família = [[Corvidae]]
| género = ''[[Garrulus]]''
| espécie = '''''G. glandarius'''''
| binomial = ''Garrulus glandarius''
| binomial_autoridade = ([[Lineu]], [[1758]])
| subdivisão_nome = Sub-espécies
| subdivisão = 
<center>''Ver t

Page [[Gaio-comum]] saved


In [31]:
#index = 2
#while index < len(species_list):
#pagename = species_list[index]
#print(pagename)
#wikicode = parse(pagename)
#templates = wikicode.filter_templates()
#print(templates)
#template = templates[1]
#print(template.name)
#print(template.params)
#print(template.get("estado").value)
#estado_ref = iucn_ref(binomial[index], api_token)
#print(estado_ref)
#estado, binomial_autoridade = iucn_estado(binomial[index], api_token)
#print(estado)
#print(binomial_autoridade)
#for template in wikicode.filter_templates():
#    if template.name.matches("Info/Taxonomia") and not template.has("estado_ref"):
#        template.add("estado_ref", estado_ref)
#    if template.name.matches("Info/Taxonomia") and not template.has("estado"):
#        template.add("estado", estado)
#    if template.name.matches("Info/Taxonomia") and not template.has("binomial_autoridade"):
#        template.add("binomial_autoridade", binomial_autoridade)
#    if template.name.matches("Info/Taxonomia") and template.has("cor"):
#        template.remove("cor")
#    if template.name.matches("Info/Taxonomia") and template.has("estado"):
#        if template.get("estado").value != estado:
#        template.replace("estado", estado)
#    if template.name.matches("Sem-fontes"):
#        wikicode.remove(template)



#wikicode = pywikibot.add_text(wikicode, "== Referências ==\n<references/>")
#print(wikicode)

#wikicode2 = iucn_insert(wikicode, estado_ref, estado, binomial_autoridade)
#estado
#print(wikicode)
#print(wikicode2)
#text = str(wikicode)
#pagename.text = text
#pagename.save()
#if not wikicode.find("<ref name=\"IUCN\"/>"):
#wikicode = re.sub(r'^((?:[A-Z]|\').+?\.)', r'\1<ref name="IUCN"/>', wikicode)
#print(wikicode)
#site = pywikibot.Site('pt', 'wikipedia')
#site
#page = pywikibot.Page(site, pagename)
#page
#text = page.get()
#text
#print(page.text)
#page_text = str(wikicode)
#page_text
#print(page_text)
#page.text = add_text("== Referências ==\n<references/>")

#ref_iucn = '<ref name=\"IUCN\"/>'
#ref_iucn_n = re.findall('<ref name=\"IUCN\"/>', page_text)
#print(ref_iucn_n)
#if ref_iucn_n == []:
#page_text.find('<ref name=\"IUCN\"/>')== -1
#if  page_text.find('<ref name=\"IUCN\"/>') == -1:
#    regex = re.compile (r'^((?:[A-Z]|\').+?\.)', re.M)
#    page_text = re.sub(regex, '\g<1><ref name=\"IUCN\"/>', page_text, re.M)
    #page_text = re.sub(r'^((?:[A-Z]|\').+?\.)', '\g<1>'+ref_iucn, page_text)
#print(page_text)

#test = NoReferencesBot.addReferences(page_text)
#bot = NoReferencesBot()
#page.text = bot.addReferences(page_text)
#print(page.text)
#page.text = page_text


#page.text.find("<ref name=\"IUCN\"\/>")
#page.text = re.sub(r'^(([A-Z]|\')[^\.]+)(\.)', r'\1\3<ref name="IUCN"/>', page.text)
#print(page.text)
#print(bot)
#save_page(wikicode, pagename)
#page.save("adicionar referência, estado e binomial autoridade baseado em IUCNredlist")
#index += 1

In [ ]:
#index = 0
#for pagename in species_list:
#    wikicode = parse(pagename)
#    estado_ref = iucn_ref(binomial[index], api_token)
#    estado, binomial_autoridade = iucn_estado(binomial[index], api_token)
#    wikicode = iucn_insert(wikicode, estado_ref, estado, binomial_autoridade)
#    save_page(wikicode, pagename)
#    index += 1

In [ ]:
#gen = pagegenerators.file('IUCNspecies.txt')
#for page in gen:
    #Do something with the page object, for example:
#    text = page.text
    #print(text)
#    replaced_text = text.replace()

#page = pywikibot.Page(site, pagename)
#page.exists()
#text = page.text
#print(text)
#replaced_text = text.replace(r'\|\s?estado\s?=\s?', '')
#help(text.replace)

#res = re.compile('\|\s?estado\s?=\s?*(\d+)').search(text)
#if res:
#    print("estado is already assign to %s" % res.group(1))
#else:
#    indexTitlePage = page.text.replace('|Number of pages=','|Number of pages='+indexPages)
#    page.save(summary='67')

In [ ]:
#import json

#tspekii_citation = '{"name":"Tragelaphus spekii","result":[{"citation":"IUCN SSC Antelope Specialist Group 2016. Tragelaphus spekii. The IUCN Red List of Threatened Species 2016: e.T22050A115164901. https://dx.doi.org/10.2305/IUCN.UK.2016-3.RLTS.T22050A50195827.en .Downloaded on 27 October 2021"}]}'
#iucn_cit_dict = json.loads(iucn_json)

#print(type(iucn_citation))
#print(type(iucn_cit_dict))
#print(iucn_cit_dict['name'])
#print(iucn_cit_dict['result'])
#print(type(iucn_cit_dict['result']))

#print(type(iucn_cit_dict['result'][0]))
#print(iucn_json['result'][0]['citation'])

In [ ]:
#index = 0
#while index < len(species_list):
    #Referência da avaliação
#    species = species_list[index]
#    citation_url = "http://apiv3.iucnredlist.org/api/v3/species/citation/" + species + "?token=" + api_token

#    iucn_get = requests.get(citation_url)
    #print(iucn_get)
#    iucn_json = iucn_get.json()
#    print(iucn_json)
    #iucn_wiki_citation = re.sub(r'(.+)\.? ([0-9]{4})\. ([A-Z][a-z]+) ([a-z]+)\. The IUCN Red List of Threatened Species ([0-9]{4}): (e\.T.+)\. https:\/\/dx\.doi\.org\/(10\.2305\/IUCN\.UK\..+\.en) \.Downloaded on ([0-9]{1,2} [A-Za-z]+ 202[1-9])', r'{{Citar IUCN doi|assessores=\1 |ano=\2 |género=\3 |espécie=\4 |anoIUCN=\5 |versão= \6|doi= \7|acessodata= 28 de outubro de 2021}}', iucn_json['result'][0]['citation'])
    #print(iucn_wiki_citation)
    
    #Estado da espécie
#    estado_url = "http://apiv3.iucnredlist.org/api/v3/species/" + species + "?token=" + api_token

#    iucn_get2 = requests.get(estado_url)
#    iucn_json2 = iucn_get2.json()
#    print(iucn_json2)

#    estado = iucn_json2['result'][0]['category']
#    print(estado)
    
    #Find {{Info/Taxonomia}}, find if it has parameter "estado ="
    #Add "estado =" if it not exists
    #Add "estado_ref=<ref name="IUCN">iucn_wiki_citation</ref> if it not exists, or empty 
    #Add "==Referências==\n<references/>" if no reference section
    #Add "<ref name="IUCN"/>" to end of first sentence
  #  index += 1

In [ ]:
#species = binomial[index]
#citation_url = "http://apiv3.iucnredlist.org/api/v3/species/citation/" + species + "?token=" + api_token

#iucn_get = requests.get(citation_url)
#print(iucn_get)
#iucn_json = iucn_get.json()
#print(iucn_json)
#iucn_wiki_citation = re.sub(r'(.+)\.? ([0-9]{4})\. ([A-Z][a-z]+) ([a-z]+)\. The IUCN Red List of Threatened Species ([0-9]{4}): (e\.T.+)\. https:\/\/dx\.doi\.org\/(10\.2305\/IUCN\.UK\..+\.en) \.Downloaded on ([0-9]{1,2} [A-Za-z]+ 202[1-9])', r'{{Citar IUCN doi|assessores=\1 |ano=\2 |género=\3 |espécie=\4 |anoIUCN=\5 |versão= \6|doi= \7|acessodata= 28 de outubro de 2021}}', iucn_json['result'][0]['citation'])
#estado_ref = iucn_wiki_citation
#print(iucn_wiki_citation)
    
#Estado da espécie
#estado_url = "http://apiv3.iucnredlist.org/api/v3/species/" + species + "?token=" + api_token

#iucn_get2 = requests.get(estado_url)
#iucn_json2 = iucn_get2.json()
#print(iucn_json2)

#estado = iucn_json2['result'][0]['category']
#print(estado)
#binomial_autoridade = iucn_json2['result'][0]['authority']
#print(binomial_autoridade)

In [ ]:
#wikicode = mwparserfromhell.parse(text)
#print(wikicode)
#templates = wikicode.filter_templates()
#print(templates)
#template = templates[0]
#print(template.name)
#print(template.params)
#print(template.get(1).value)
#print(template.get("eggs").value)

#for template in wikicode.filter_templates():
#    if template.name.matches("Info/Taxonomia") and not template.has("estado"):
#        template.add("estado", estado)
#        template.add("estado_ref", estado_ref)
#    elif template.name.matches("Info/Taxonomia") and not template.has("binomial_autoridade"):
#        template.add("binomial_autoridade", binomial_autoridade)
#    elif template.name.matches("Info/Taxonomia") and template.has("estado") and not template.has("estado"):
#        template.add("estado_ref", estado_ref)
#template.remove("sem-fontes")
        
#print(wikicode)
#newtext = str(wikicode)
